In [ ]:
import rdp
import numpy as np
import re
from PIL import Image
import h5py
from matplotlib import pyplot as plt
from matplotlib import dates
from sklearn.cluster import (KMeans, SpectralClustering, AgglomerativeClustering, DBSCAN, OPTICS, Birch, MeanShift,
AffinityPropagation)
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.metrics import mean_squared_error as mse, silhouette_score
from glob import glob
from peakfinder import detect_peaks
from math import floor, ceil
from itertools import combinations
from datetime import datetime
from tqdm import tqdm
from features import *
from visualisation import *
np.set_printoptions(suppress=False)

In [ ]:
wav, soundings, shift_std, shift_mean, space, time = load_soundings("../data/level_1p0a/*[I|E].h5", smoothing="poly")

In [ ]:
centred_soundings = centre(soundings)

In [ ]:
comp_wav, comp_centred_soundings = dropout(wav, centred_soundings, 11)

In [ ]:
grad, polyres, poly_coeffs, curv, gradf = create_features(wav, centred_soundings, standardize, "grad", "polyres", "poly_coeffs", "curv", "gradf")

In [ ]:
logpolyres = np.log(polyres)/np.log(polyres).std(axis=0)
cbrt_poly_coeffs = np.cbrt(poly_coeffs)
cbrt_grad = np.cbrt(grad)
cbrt_curv = np.cbrt(curv)

In [ ]:
eng_features = np.hstack([poly_coeffs, polyres, grad, curv])
att_eng_features = np.hstack([cbrt_poly_coeffs, logpolyres, cbrt_grad, cbrt_curv])

In [ ]:
standardised_combined_features = np.hstack([comp_centred_soundings/comp_centred_soundings.std(axis=0), eng_features])
att_standardised_combined_features = np.hstack([comp_centred_soundings/comp_centred_soundings.std(axis=0), att_eng_features])

In [ ]:
index = {"poly_coeffs3": poly_coeffs[:,0], "poly_coeffs2": poly_coeffs[:,1], "poly_coeffs1": poly_coeffs[:,2], 
         "polyres1": polyres[:,0], "polyres2": polyres[:,1], "grad1": grad[:,0], "grad2": grad[:,1], 
         "grad3": grad[:,2], "grad4": grad[:,3], "curv1": curv[:,0], "curv2": curv[:,1], 
         "logpolyres1": logpolyres[:,0], "logpolyres2": logpolyres[:,1], "cbrt_poly_coeffs3": cbrt_poly_coeffs[:,0], 
         "cbrt_poly_coeffs2": cbrt_poly_coeffs[:,1], "cbrt_poly_coeffs1": cbrt_poly_coeffs[:,2], 
         "cbrt_grad1": cbrt_grad[:,0], "cbrt_grad2": cbrt_grad[:,1], "cbrt_grad3": cbrt_grad[:,2], 
         "cbrt_grad4": cbrt_grad[:,3], "cbrt_curv1": cbrt_curv[:,0], "cbrt_curv2": cbrt_curv[:,1]}

In [ ]:
m1 = get_mask([polyres[:,1], 5, 10])
m2 = get_mask([polyres[:,0], 4, 7])

In [ ]:
soundings[m2|m1].shape

In [ ]:
view_soundings(wav, soundings[m1])

In [ ]:
lat = space[m1][:,1]
lon = space[m1][:,2]

In [ ]:
marsImage = Image.open("../mars.png")
fig = plt.figure(figsize=(15,9))
ax = fig.add_subplot()
ax.grid()
ax.set_title("Locations of highly periodic soundings")
ax.set_xticks(range(-180, 181, 30))
ax.set_yticks(range(-90, 91, 30))
ax.set_xlim(-180, 180)
ax.set_ylim(-90, 90)
    
ax.scatter(lon, lat)
ax.imshow(marsImage, extent=[-180, 180, 90, -90]);

In [ ]:
plt.hist(space[:,0][m1])

In [ ]:
plt.hist(gradf[:,3])

In [ ]:
m5 = gradf[:,3] > 4
soundings[m5].shape
#view_soundings(wav, soundings[m5])

In [ ]:
# dminmax and ddminmax < 0
m3 = gradf[:,0] < 0
m4 = gradf[:,2] < 0
soundings[m4&m3].shape

In [ ]:
# choose number of optima
mm = (optimum_counter(soundings[m3&m4], 10)) == 2
soundings[m3&m4][mm.flatten()].shape
view_soundings(wav, soundings[m3&m4][mm.flatten()][::2])

In [ ]:
dsoundings = np.gradient(soundings, axis=1, edge_order=2)
idx = abs(dsoundings).argmax(axis=1)

In [ ]:
dsoundings[np.arange(dsoundings.shape[0]),idx][0]

In [ ]:
def gradient_features(soundings):
    """
    Calculate derivative and second derivative of smoothed soundings and use these to calculate further features 
    of soundings

    Inputs
    soundings     array of soundings of shape soundings[sounding,wavelength]

    Outputs 
    dminmax       minimum gradient multiplied by maximum in gradient
    dstd          standard deviation of gradient
    ddminmax      minimum and maximum of second derviative multiplied together
    ddstd         standard deviation of curvature
    """
    dsoundings = np.gradient(soundings, axis=1, edge_order=2)
    ddsoundings = np.gradient(dsoundings, axis=1, edge_order=2)
    dminmax = dsoundings.max(axis=1) * dsoundings.min(axis=1)
    dstd = dsoundings.std(axis=1)
    ddminmax = ddsoundings.max(axis=1) * ddsoundings.min(axis=1)
    ddstd = ddsoundings.std(axis=1)

    return np.vstack([dminmax/dminmax.std(), dstd/dstd.std(), ddminmax/ddminmax.std(), ddstd/ddstd.std()]).T